In [1]:
named_place_name_path = '../../data/bhp/named_place_name.csv'

In [2]:
import os, datetime
import pandas as pd

import geovpylib.utils as u
import geovpylib.analysis as a
import geovpylib.database as db

db.connect_external(os.environ.get('YELLOW_BHP'))
eta = u.Eta()

>> Connecting to PGSQL Database ... Connected!


## Fetch full table

In [3]:
named_place_name = db.query('select * from bhp.named_place_name')

## Filter unwanted columns

According to the wiki page, we can get rid of those columns:

In [4]:
named_place_name.drop(columns=['complement_address'], inplace=True)
named_place_name.drop(columns=['name_type'], inplace=True)

## Remove binary characters

In [5]:
named_place_name.replace({'\r\n':''}, regex=True, inplace=True)
named_place_name.replace({'<p>':''}, regex=True, inplace=True)
named_place_name.replace({'</p>':''}, regex=True, inplace=True)

u.remove_binary_chars(named_place_name)

## Discovery

In [6]:
a.discover(named_place_name, uniq_ex_nb=3)

Columns contain:
Total number of rows: 128529
  -    "pk_named_place_name":   0.00% empty - 128529 (100.00%) uniques (eg: 152213; 152242; 152214)
  -          "creation_time":   0.00% empty -   4599 (  3.58%) uniques (eg: 2012-07-13...; 2012-09-10...; 2012-07-13...)
  -         "fk_named_place":   0.00% empty - 127466 ( 99.17%) uniques (eg: 120967; 48; 90320)
  -                  "notes":   0.00% empty -    769 (  0.60%) uniques (eg: None; Nom donné ...; Nom offici...)
  -     "comment_begin_year":   0.00% empty -     24 (  0.02%) uniques (eg: None; ; fondation )
  -               "lang_iso":   0.00% empty -     36 (  0.03%) uniques (eg: None; spa; fra)
  -       "comment_end_year":   0.00% empty -     37 (  0.03%) uniques (eg: None; ; devient Ma...)
  -       "is_standard_name":   0.00% empty -      2 (  0.00%) uniques (eg: True; False)
  -                   "name":   0.00% empty -  85475 ( 66.50%) uniques (eg: Hellemmes; Espagne; Col de Par...)
  -            "concat_plna":   0.00% e

## Type parsing

According to the table before, we will parse each column by the most meaningful type.

In [7]:
a.set_types(named_place_name, {
   "pk_named_place_name": "int",
           "concat_plna": "string",
      "is_standard_name": "boolean",
        "fk_named_place": "int",
         "creation_time": "datetime",
               "creator": "int",
              "modifier": "int",
                  "name": "string",
              "lang_iso": "string",
     "modification_time": "datetime",
                 "notes": "string",
                "number": "int",
     "complement_number": "string",
"fk_abob_napl_name_type": "int",
    "comment_begin_year": "string",
      "comment_end_year": "string",
            "begin_year": "int",
              "end_year": "int",
           "begin_month": "int",
             "begin_day": "int",
             "end_month": "int",
               "end_day": "int",
})     

# So that they appear correctly
a.set_types(named_place_name, {
        "begin_year": 'string',
          "end_year": 'string',
       "begin_month": 'string',
         "end_month": 'string',
         "begin_day": 'string',
           "end_day": 'string',
})

## Handle data

### begin and end dates

We create 2 new columns, made of the joining of `begin_year`, `begin_month`, `begin_day` and `end_year`, `end_month`, `end_day`.

In [8]:
def prefix_date(date):
    if pd.isna(date): return date
    if len(str(date)) == 3: return f'0{date}'
    return date 

# Set the length of begin_year and end_year to 4
named_place_name['begin_year'] = [prefix_date(d) for d in named_place_name['begin_year']]
named_place_name['end_year'] = [prefix_date(d) for d in named_place_name['end_year']]

# Replace existing 6 columns by 2
named_place_name['begin_date'] = named_place_name['begin_year'] + named_place_name['begin_month'] + named_place_name['begin_day']
named_place_name['end_date'] = named_place_name['end_year'] + named_place_name['end_month'] + named_place_name['end_day']
named_place_name.drop(columns=['begin_year', 'begin_month', 'begin_day', 'end_year', 'end_month', 'end_day'], inplace=True)

# Parse into datetime
named_place_name['begin_date'] = [datetime.datetime.strptime(d, '%Y%m%d') if pd.notna(d) else pd.NaT for d in named_place_name['begin_date']]
named_place_name['end_date'] = [datetime.datetime.strptime(d, '%Y%m%d') if pd.notna(d) else pd.NaT for d in named_place_name['end_date']]

### lang_iso

Some cleaning is made on this column, in order to fit ISO639-2/T (3 letters code, native prefered, eg 'deu' instead of 'ger').

In [9]:
named_place_name['lang_iso'].replace('   ', pd.NA, inplace=True)
named_place_name['lang_iso'].replace('Fra', 'fra', inplace=True)
named_place_name['lang_iso'].replace('FRA', 'fra', inplace=True)
named_place_name['lang_iso'].replace('Ita', 'ita', inplace=True)
named_place_name['lang_iso'].replace('fr ', 'fra', inplace=True)
named_place_name['lang_iso'].replace('und', pd.NA, inplace=True)

## Save data

In [10]:
u.save_df(named_place_name, named_place_name_path)